In [1]:
from IPython.display import HTML, display

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

import pandas as pd
import numpy as np
import ast
import random
import math 
import time
import sys 

data = pd.read_csv("Concrete_Data.csv")


In [2]:
x = data.iloc[:,0:8]
y = data.iloc[:,8:]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 100) 


X_train = sm.add_constant(x_train) ## let's add an intercept (beta_0) to our model
X_test = sm.add_constant(x_test) 

lm2 = sm.OLS(y_train,X_train).fit()

print(lm2.summary())

y_pred2 = lm2.predict(X_test) 

                            OLS Regression Results                            
Dep. Variable:                    CMS   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.609
Method:                 Least Squares   F-statistic:                     161.0
Date:                Fri, 11 Oct 2019   Prob (F-statistic):          4.37e-162
Time:                        11:25:20   Log-Likelihood:                -3090.4
No. Observations:                 824   AIC:                             6199.
Df Residuals:                     815   BIC:                             6241.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -34.2735     29.931  

C:\Users\okano\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [3]:
#Detecting Outliers:
influence = lm2.get_influence()  
resid_student = influence.resid_studentized_external

resid = pd.concat([x_train,pd.Series(resid_student,name = "Studentized Residuals")],axis = 1)
print(resid.head())

#remove studentized residuals > 3
resid.loc[np.absolute(resid["Studentized Residuals"]) > 3,:]

#get index values
ind = resid.loc[np.absolute(resid["Studentized Residuals"]) > 3,:].index

#remove from data
y_train.drop(ind,axis = 0,inplace = True)
x_train.drop(ind,axis = 0,inplace = True)  #Interept column is not there
X_train.drop(ind,axis = 0,inplace = True)  #Intercept column is there



   Cement  Blast  Fly Ash  Water  Superplasticizer      CA     FA    Age  \
0   540.0    0.0      0.0  162.0               2.5  1040.0  676.0   28.0   
1   540.0    0.0      0.0  162.0               2.5  1055.0  676.0   28.0   
2   332.5  142.5      0.0  228.0               0.0   932.0  594.0  270.0   
3   332.5  142.5      0.0  228.0               0.0   932.0  594.0  365.0   
4   198.6  132.4      0.0  192.0               0.0   978.4  825.5  360.0   

   Studentized Residuals  
0               1.559672  
1              -0.917354  
2               1.057443  
3               0.637504  
4              -1.170290  


C:\Users\okano\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [4]:
#print vif values
[print(variance_inflation_factor(x_train.values, j)) for j in range(x_train.shape[1])]

15.47758260195686
3.2696650121931814
4.129325501299344
82.21008475163109
5.21853674386234
85.86694548901554
71.81633694293068
1.6861600968467656


[None, None, None, None, None, None, None, None]

In [5]:
def calculate_vif(x):
    thresh = 5.0
    output = pd.DataFrame()
    k = x.shape[1]
    vif = [variance_inflation_factor(x.values, j) for j in range(x.shape[1])]
    for i in range(1,k):
        print("Iteration no.")
        print(i)
        print(vif)
        a = np.argmax(vif)
        print("Max VIF is for variable no.:")
        print(a)
        if vif[a] <= thresh :
            break
        if i == 1 :          
            output = x.drop(x.columns[a], axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
        elif i > 1 :
            output = output.drop(output.columns[a],axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
    return(output)



train_out = calculate_vif(x_train) 

print(train_out.head())

Iteration no.
1
[15.47758260195686, 3.2696650121931814, 4.129325501299344, 82.21008475163109, 5.21853674386234, 85.86694548901554, 71.81633694293068, 1.6861600968467656]
Max VIF is for variable no.:
5
Iteration no.
2
[14.517486362670928, 3.2477734890453647, 3.968695653417151, 71.53530428408644, 5.1775267752249094, 48.27016091702854, 1.68612563105104]
Max VIF is for variable no.:
3
Iteration no.
3
[9.385732352700503, 2.0828769946572407, 3.009516852485082, 2.910827525646028, 14.418586504418215, 1.572151969798833]
Max VIF is for variable no.:
4
Iteration no.
4
[2.6936535297265767, 1.5282892036607625, 1.904439437889966, 2.890705249212633, 1.5380724272424997]
Max VIF is for variable no.:
3
     Cement  Blast  Fly Ash  Superplasticizer  Age
337   275.1    0.0    121.4               9.9   56
384   516.0    0.0      0.0               8.2   28
805   393.0    0.0      0.0               0.0   90
682   183.9  122.6      0.0               0.0   28
329   246.8    0.0    125.1              12.0    3


In [6]:
x_test.drop(["Water","CA","FA"],axis = 1,inplace = True)

Train_out = sm.add_constant(train_out) ## let's add an intercept (beta_0) to our model
X_test = sm.add_constant(x_test)
lm2 = sm.OLS(y_train,Train_out).fit()
print(lm2.summary())

[print(variance_inflation_factor(train_out.values, j)) for j in range(train_out.shape[1])]

y_pred3 = lm2.predict(X_test) 

                            OLS Regression Results                            
Dep. Variable:                    CMS   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     216.3
Date:                Fri, 11 Oct 2019   Prob (F-statistic):          6.88e-147
Time:                        11:30:45   Log-Likelihood:                -3128.8
No. Observations:                 823   AIC:                             6270.
Df Residuals:                     817   BIC:                             6298.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -11.1119      1.915  

In [7]:
print(metrics.mean_absolute_error(y_test, y_pred2))
print(metrics.mean_squared_error(y_test, y_pred2))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))

print(metrics.mean_absolute_error(y_test, y_pred3))
print(metrics.mean_squared_error(y_test, y_pred3))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred3)))

8.262654160599253
113.17875937789917
10.638550623928955
8.62405760881035
124.42658466679039
11.15466649733601


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 100) 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(x_train)
# Apply transform to both the training set and the test set.
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)

pca.fit(x_train)

x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

X_train = sm.add_constant(x_train) ## let's add an intercept (beta_0) to our model
X_test = sm.add_constant(x_test) 

lm4 = sm.OLS(y_train,X_train).fit()

print(lm4.summary())

y_pred4 = lm4.predict(X_test) 

print(metrics.mean_absolute_error(y_test, y_pred4))
print(metrics.mean_squared_error(y_test, y_pred4))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred4)))


                            OLS Regression Results                            
Dep. Variable:                    CMS   R-squared:                       0.557
Model:                            OLS   Adj. R-squared:                  0.554
Method:                 Least Squares   F-statistic:                     171.2
Date:                Fri, 11 Oct 2019   Prob (F-statistic):          9.34e-141
Time:                        11:33:09   Log-Likelihood:                -3145.6
No. Observations:                 824   AIC:                             6305.
Df Residuals:                     817   BIC:                             6338.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         35.9527      0.385     93.367      0.0